In [7]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import math
import matplotlib.pyplot as plt
import scipy
from scipy import linalg
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange, cuda, float32, float64, complex64, complex128, types
import numba as nb

# locals
from hamiltonian_generation import make_H4_sparse

# Macros and physical constants 
np.random.seed(0)
K = 13 # number of fermionic modes
J = 4 # ~"energy scale"
Q = 4 # order of coupling
N = 2*K # number of fermions
N_DIM = 2**K # Hilbert space dimensions

N_SAMPLES = 100 # number of samples to generate
N_JOBS = 20 # number of jobs to run in parallel

sigma_j = np.sqrt((J**2)*np.math.factorial(Q-1)/(N**(Q-1)))
js_test = np.random.normal(0, sigma_j, size=tuple([N for i in range(Q)]))

In [8]:
H4_func = make_H4_sparse(K, J, random_seed=0, precompute_pairs = True, precompute_quads = True)

With K=12, J=4, Q=4...


Takes Python 3.8 seconds to create the entire Hamiltonian *function* (which includes precomputing the annihilation/creation operators, the fermion operators, the pairwise and quadwise inner-products)

Takes Mathematica 26.6 seconds (7.656 per 'Timing') to precompute the annihilation/creation operators AND fermion operators, 1.3 seconds (0.3125 per 'Timing') to compute pairwise inner-products, and 14.19 seconds (2.219 per 'Timing') to compute the quadwise inner-products. 

In [9]:
tic = time.time()
H4_test = H4_func(js_test)
toc = time.time()
duration = toc - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

Duration: 8.0 minutes, 14.545088291168213 seconds


With K=12, J=4, Q=4...


Takes Python 108.8 seconds to compute Hamiltonian (with all pairwise and quadwise inner-products precomputed)

Takes Mathematica 129.4 seconds (33.265 per 'Timing') seconds to do the same. 

In [11]:
#H4_test = H4_test.toarray()
print(linalg.ishermitian(H4_test))
print(np.allclose(H4_test, np.real(H4_test)))
eigvals = np.linalg.eigvalsh(H4_test)

True
False


In [ ]:
plt.hist(eigvals, bins=30)